# PLoS_methods_bio  split

In [35]:
import os
import argparse
import time
import random
import shutil

from pathlib import Path
from math import ceil

from somenlp.utils import get_time_marker
from somenlp.feature_engineering import calculate_features_parallel


import os
from os import listdir 
from collections import Counter

import os
from os import listdir 
from collections import Counter


in_path = '/home/beck/Desktop/SoMeNLP/data/PLoS_methods_bio/'
out_path = '/home/beck/Desktop/SoMeNLP/data/PLoS_methods/'

def split_data(in_path, out_path, file_ext =".data.txt", 
               ratio =[60, 20, 20], set_names = ['train', 'devel', 'test']):
    
    in_path = in_path.rstrip('/')
    out_path = out_path.rstrip('/')

    if not os.path.isdir(in_path):
        raise(RuntimeError("Input path does not exist"))
        
    if not os.path.isdir(out_path):
        os.mkdir(out_path)
    
    if sum(ratio) != 100:
        raise(RuntimeError("Input ratio {} does not sum to 100".format(ratio)))
    if len(ratio) != len(set_names):
        raise(RuntimeError("Number of ratios and setnames has to match: {} vs {}".format(ratio, set_names)))

    #print("Loading files")
    single_files = list(Path(in_path).rglob('*{}'.format(file_ext)))
    
    all_files = []
    for entry in single_files:
        
        base_file_name = entry.name.split(file_ext)[0]
        
        base_file_entries = list(Path(in_path).rglob('{}*'.format(base_file_name)))
        
        all_files.append(base_file_entries)
    
    random.seed()
    random.shuffle(all_files)

    #print("Copying files")
    cut_sum = 0
    prev_cut_idx = 0
    for cut, name in zip(ratio, set_names):
        cut_sum += cut
        cut_idx = ceil(len(all_files) * cut_sum / 100) 
        
        out_folder_name = in_path.rsplit('/')[-1]
        new_output_location = '{}/{}_{}'.format(out_path.rstrip('/'), out_folder_name, name) 
        
        if not os.path.isdir(new_output_location):
            os.makedirs(new_output_location)
            
        for files in all_files[prev_cut_idx:cut_idx]:
            for f in files:
                source_path = str(f)
                target_path = '{}/{}'.format(new_output_location, f.name)
                shutil.copy(source_path, target_path)
        prev_cut_idx = cut_idx
    
    print("Done")
    

def purposeLabel_counter(path):
    
    import os
    from os import listdir 
    from collections import Counter
    
    
    def list_file_names(path, file_ext='.labels.txt'):
    
        file_names_list = []

        for file_name in os.listdir(path):
            if not file_name.endswith(file_ext): continue
            file_names_list.append(file_name) 

        file_names_list.sort()

        return file_names_list
    
    file_name_list = list_file_names(path)
    
    interest_list = ["Analysis", "Modelling", "Stimulation", "DataCollection", "DataPreProcss", 
                 "Simulation", "Visualization", "Programming"]
    
    all_purpose_labels = []
    
    for file_name in file_name_list[:]:
        file_path = path + file_name
        
        with open(file_path, 'r') as f:
            
            list_lines = f.readlines()
            
            #print(list_of_lebs)
            list_of_tokens = ' '.join(list_lines).split()
            
            for tok in list_of_tokens:
                if tok != 'O' and (len(tok.split('-')[1].split('_')) == 3 ):
                    if tok.split('-')[1].split('_')[2] in interest_list:
                        
                        purpose = tok.split('-')[1].split('_')[2]
                        
                        all_purpose_labels.append(purpose)
    
                        
    return  dict(Counter(all_purpose_labels))

def train_test_dev(whole_path, train_path, test_path, dev_path):
    
    _whole = purposeLabel_counter(whole_path)
    _train = purposeLabel_counter(train_path)
    _test  = purposeLabel_counter(test_path)
    _dev   = purposeLabel_counter(dev_path)
    
    
    
    _whole_sorted = dict(sorted(_whole.items()))
    _train_sorted = dict(sorted(_train.items()))
    _test_sorted  = dict(sorted(_test.items()))
    _dev_sorted   = dict(sorted(_dev.items()))
    
    return _whole_sorted, _train_sorted, _test_sorted, _dev_sorted

def __proportion_calculator_with_key(whole_dict, train_dict, test_dict, dev_dict ):
    
    train_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), train_dict.items()):
        if k2 == k: 
            train_percentage = int(v2 / v * 100)
            train_purpose_props.append((train_percentage, k))
            
    dev_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), dev_dict.items()):
        if k2 == k:
            
            dev_percentage = int(v2 / v * 100)
            dev_purpose_props.append((dev_percentage,k))
            
    test_purpose_props = []       
    for (k,v), (k2,v2) in zip(whole_dict.items(), test_dict.items()):
        if k2 == k:
            
            test_percentage = int(v2 / v * 100)
            test_purpose_props.append((test_percentage, k))
            
    return train_purpose_props, dev_purpose_props, test_purpose_props


def __proportion_calculator(whole_dict, train_dict, test_dict, dev_dict ):
    
    train_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), train_dict.items()):
        if k2 == k: 
            train_percentage = int(v2 / v * 100)
            train_purpose_props.append(train_percentage)
            
    dev_purpose_props = []
    for (k,v), (k2,v2) in zip(whole_dict.items(), dev_dict.items()):
        if k2 == k:
            
            dev_percentage = int(v2 / v * 100)
            dev_purpose_props.append(dev_percentage)
            
    test_purpose_props = []       
    for (k,v), (k2,v2) in zip(whole_dict.items(), test_dict.items()):
        if k2 == k:
            
            test_percentage = int(v2 / v * 100)
            test_purpose_props.append(test_percentage)
            
    return train_purpose_props, dev_purpose_props, test_purpose_props



def _within_range(list_):
    
    bool_list = []
    for val in list_:
        if val in range(54,66):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
    return all(bool_list)   # apply & to all and return


def _within_range_pubmed(list_):
    
    bool_list = []
    for val in list_:
        if val in range(52,68):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
            
    withn_range_bool = all(bool_list) # apply & to all and return
    
    # check if all class labels are included , class label stimulation is missing in the PubMed 
    if (len(bool_list) == 7):
        all_classes_included = True
    else:
        all_classes_included = False
        
    result = all_classes_included and withn_range_bool
            
    return result   


def _within_range_dev_test(list_):
    
    bool_list = []
    for val in list_:
        if val in range(15,25):
            bool_list.append(True)
        else:
            bool_list.append(False)
            
    return all(bool_list)   # apply & to all and return

In [6]:
#split_data(in_path, out_path)

# plos methods bio   train , test , dev

#path_whole = '/home/beck/Desktop/SoMeNLP/data/PLoS_methods_bio/'
#path_train = "/home/beck/Desktop/SoMeNLP/data/PLoS_methods/PLoS_methods_bio_train/" 
#path_test = "/home/beck/Desktop/SoMeNLP/data/PLoS_methods/PLoS_methods_bio_test/" 
#path_dev = "/home/beck/Desktop/SoMeNLP/data/PLoS_methods/PLoS_methods_bio_devel/"
    
    
#calculate the new distribution 
#whole_dict, train_dict, test_dict, dev_dict = train_test_dev(path_whole, path_train, path_test, path_dev)


# calculate the new proportion
#train_props, dev_props, test_props = __proportion_calculator(whole_dict, train_dict, test_dict, dev_dict)

#split_result = _within_range(train_props)

#i = 0

#while not(_within_range(train_props)):
    
    #print(f'splitting ... loop {i}')
    
    #split_data(in_path, out_path)
    
    #i += 1
    
    
    #calculate the new distribution 
    #whole_dict, train_dict, test_dict, dev_dict = train_test_dev(path_whole, path_train, path_test, path_dev)
    
    
    # calculate the new proportion
    #train_props, dev_props, test_props = __proportion_calculator(whole_dict, train_dict, test_dict, dev_dict)
    
    #split_result = _within_range(train_props)
    
    #print('is the split fair ?')
    
    #if split_result:
        
        #print('split is balanced. stopped splitting')
        
    #else:
        
        #shutil.rmtree(path_train, ignore_errors=False, onerror=None)
        #shutil.rmtree(path_test, ignore_errors=False, onerror=None)
        #shutil.rmtree(path_dev, ignore_errors=False, onerror=None)
        
        #print(f'split is not yet balanced {train_props}.  splitting further ...')
        #print()
        
        
    
    #break

Done
splitting ... loop 0
Done
split is not yet balanced [84, 82, 72, 81, 98, 91, 81, 87].  splitting further ...

splitting ... loop 1
Done
split is not yet balanced [64, 51, 75, 65, 71, 45, 78, 47].  splitting further ...

splitting ... loop 2
Done
split is not yet balanced [57, 61, 52, 62, 79, 54, 47, 47].  splitting further ...

splitting ... loop 3
Done
split is not yet balanced [61, 60, 51, 47, 42, 41, 53, 74].  splitting further ...

splitting ... loop 4
Done
split is not yet balanced [65, 55, 71, 66, 57, 75, 71, 66].  splitting further ...

splitting ... loop 5
Done
split is not yet balanced [62, 62, 72, 61, 67, 54, 67, 78].  splitting further ...

splitting ... loop 6
Done
split is not yet balanced [62, 55, 63, 50, 70, 58, 26, 58].  splitting further ...

splitting ... loop 7
Done
split is not yet balanced [63, 60, 60, 63, 65, 83, 59, 48].  splitting further ...

splitting ... loop 8
Done
split is not yet balanced [58, 56, 67, 67, 79, 45, 69, 59].  splitting further ...

split

Done
split is not yet balanced [59, 58, 66, 54, 45, 66, 63, 53].  splitting further ...

splitting ... loop 75
Done
split is not yet balanced [56, 69, 44, 60, 68, 79, 59, 64].  splitting further ...

splitting ... loop 76
Done
split is not yet balanced [55, 62, 65, 57, 59, 66, 64, 57].  splitting further ...

splitting ... loop 77
Done
split is not yet balanced [58, 55, 55, 57, 56, 54, 56, 47].  splitting further ...

splitting ... loop 78
Done
split is not yet balanced [62, 67, 67, 57, 60, 50, 57, 59].  splitting further ...

splitting ... loop 79
Done
split is not yet balanced [66, 64, 65, 68, 62, 54, 69, 58].  splitting further ...

splitting ... loop 80
Done
split is not yet balanced [60, 64, 54, 58, 45, 45, 54, 53].  splitting further ...

splitting ... loop 81
Done
split is not yet balanced [61, 68, 72, 60, 67, 54, 71, 74].  splitting further ...

splitting ... loop 82
Done
split is not yet balanced [64, 64, 66, 68, 73, 79, 59, 76].  splitting further ...

splitting ... loop 83
D

Done
split is not yet balanced [58, 63, 60, 63, 87, 37, 70, 70].  splitting further ...

splitting ... loop 149
Done
split is not yet balanced [63, 61, 66, 62, 40, 75, 61, 61].  splitting further ...

splitting ... loop 150
Done
split is not yet balanced [55, 56, 58, 60, 53, 66, 70, 65].  splitting further ...

splitting ... loop 151
Done
split is not yet balanced [64, 58, 67, 72, 50, 83, 43, 70].  splitting further ...

splitting ... loop 152
Done
split is not yet balanced [61, 64, 74, 66, 60, 45, 64, 50].  splitting further ...

splitting ... loop 153
Done
split is not yet balanced [58, 54, 61, 53, 54, 75, 64, 64].  splitting further ...

splitting ... loop 154
Done
split is not yet balanced [61, 52, 63, 68, 56, 79, 70, 61].  splitting further ...

splitting ... loop 155
Done
split is not yet balanced [64, 56, 48, 54, 64, 75, 61, 39].  splitting further ...

splitting ... loop 156
Done
split is not yet balanced [62, 64, 52, 62, 29, 62, 38, 60].  splitting further ...

splitting ... l

Done
split is not yet balanced [61, 62, 61, 53, 65, 83, 87, 66].  splitting further ...

splitting ... loop 223
Done
split is not yet balanced [58, 69, 59, 71, 37, 54, 66, 44].  splitting further ...

splitting ... loop 224
Done
split is not yet balanced [63, 64, 62, 57, 59, 66, 66, 48].  splitting further ...

splitting ... loop 225
Done
split is not yet balanced [53, 58, 52, 57, 39, 50, 59, 69].  splitting further ...

splitting ... loop 226
Done
split is not yet balanced [63, 65, 55, 56, 68, 62, 47, 55].  splitting further ...

splitting ... loop 227
Done
split is not yet balanced [67, 60, 67, 78, 76, 54, 47, 52].  splitting further ...

splitting ... loop 228
Done
split is not yet balanced [64, 62, 67, 65, 75, 45, 36, 58].  splitting further ...

splitting ... loop 229
Done
split is not yet balanced [57, 58, 55, 59, 50, 20, 57, 43].  splitting further ...

splitting ... loop 230
Done
split is not yet balanced [52, 61, 47, 50, 81, 70, 66, 57].  splitting further ...

splitting ... l

Done
split is not yet balanced [60, 47, 54, 58, 65, 41, 50, 65].  splitting further ...

splitting ... loop 297
Done
split is not yet balanced [50, 55, 47, 49, 29, 50, 59, 61].  splitting further ...

splitting ... loop 298
Done
split is not yet balanced [64, 61, 71, 56, 59, 25, 61, 60].  splitting further ...

splitting ... loop 299
Done
split is not yet balanced [64, 65, 56, 60, 82, 54, 64, 52].  splitting further ...

splitting ... loop 300
Done
split is not yet balanced [55, 55, 52, 42, 54, 66, 57, 58].  splitting further ...

splitting ... loop 301
Done
split is not yet balanced [60, 57, 62, 57, 50, 62, 69, 60].  splitting further ...

splitting ... loop 302
Done
split is not yet balanced [58, 54, 49, 55, 73, 37, 76, 68].  splitting further ...

splitting ... loop 303
Done
split is not yet balanced [51, 54, 57, 68, 85, 62, 57, 57].  splitting further ...

splitting ... loop 304
Done
split is not yet balanced [56, 59, 51, 60, 71, 50, 47, 52].  splitting further ...

splitting ... l

Done
split is not yet balanced [53, 70, 52, 47, 62, 58, 73, 59].  splitting further ...

splitting ... loop 371
Done
split is not yet balanced [70, 70, 76, 66, 79, 58, 76, 60].  splitting further ...

splitting ... loop 372
Done
split is not yet balanced [64, 60, 67, 59, 53, 62, 59, 52].  splitting further ...

splitting ... loop 373
Done
split is not yet balanced [67, 49, 65, 57, 50, 45, 61, 46].  splitting further ...

splitting ... loop 374
Done
split is not yet balanced [57, 65, 57, 45, 73, 66, 56, 51].  splitting further ...

splitting ... loop 375
Done
split is not yet balanced [61, 61, 68, 62, 35, 50, 77, 69].  splitting further ...

splitting ... loop 376
Done
split is not yet balanced [58, 63, 57, 61, 57, 33, 70, 57].  splitting further ...

splitting ... loop 377
Done
split is not yet balanced [66, 62, 67, 67, 43, 54, 61, 64].  splitting further ...

splitting ... loop 378
Done
split is not yet balanced [63, 73, 69, 64, 85, 58, 56, 73].  splitting further ...

splitting ... l

Done
split is not yet balanced [53, 52, 51, 51, 46, 45, 52, 62].  splitting further ...

splitting ... loop 445
Done
split is not yet balanced [64, 51, 72, 57, 71, 50, 59, 69].  splitting further ...

splitting ... loop 446
Done
split is not yet balanced [53, 60, 52, 59, 60, 87, 73, 44].  splitting further ...

splitting ... loop 447
Done
split is not yet balanced [58, 48, 56, 60, 73, 66, 49, 62].  splitting further ...

splitting ... loop 448
Done
split is not yet balanced [57, 59, 57, 66, 26, 58, 61, 61].  splitting further ...

splitting ... loop 449
Done
split is not yet balanced [63, 67, 53, 62, 60, 70, 49, 49].  splitting further ...

splitting ... loop 450
Done
split is not yet balanced [58, 53, 64, 64, 54, 75, 66, 50].  splitting further ...

splitting ... loop 451
Done
split is not yet balanced [53, 53, 54, 55, 76, 75, 59, 41].  splitting further ...

splitting ... loop 452
Done
split is not yet balanced [62, 56, 59, 62, 53, 50, 71, 48].  splitting further ...

splitting ... l

Done
split is not yet balanced [61, 69, 68, 73, 42, 50, 56, 68].  splitting further ...

splitting ... loop 519
Done
split is not yet balanced [59, 60, 72, 66, 56, 45, 63, 70].  splitting further ...

splitting ... loop 520
Done
split is not yet balanced [70, 65, 78, 65, 68, 75, 63, 46].  splitting further ...

splitting ... loop 521
Done
split is not yet balanced [59, 58, 54, 62, 46, 33, 73, 62].  splitting further ...

splitting ... loop 522
Done
split is not yet balanced [55, 55, 47, 52, 81, 41, 87, 60].  splitting further ...

splitting ... loop 523
Done
split is not yet balanced [60, 60, 57, 60, 60, 75, 29, 70].  splitting further ...

splitting ... loop 524
Done
split is not yet balanced [65, 57, 55, 57, 78, 70, 81, 62].  splitting further ...

splitting ... loop 525
Done
split is not yet balanced [63, 52, 58, 62, 75, 62, 50, 47].  splitting further ...

splitting ... loop 526
Done
split is not yet balanced [57, 61, 52, 51, 51, 79, 56, 50].  splitting further ...

splitting ... l

Done
split is not yet balanced [66, 59, 69, 69, 43, 54, 80, 48].  splitting further ...

splitting ... loop 593
Done
split is not yet balanced [63, 56, 72, 77, 85, 41, 56, 71].  splitting further ...

splitting ... loop 594
Done
split is not yet balanced [55, 60, 56, 48, 51, 62, 78, 68].  splitting further ...

splitting ... loop 595
Done
split is not yet balanced [65, 67, 66, 63, 53, 70, 56, 51].  splitting further ...

splitting ... loop 596
Done
split is not yet balanced [62, 70, 81, 63, 46, 62, 59, 67].  splitting further ...

splitting ... loop 597
Done
split is not yet balanced [53, 57, 56, 47, 64, 62, 70, 56].  splitting further ...

splitting ... loop 598
Done
split is not yet balanced [65, 57, 60, 66, 51, 75, 76, 58].  splitting further ...

splitting ... loop 599
Done
split is not yet balanced [60, 59, 59, 68, 32, 45, 70, 55].  splitting further ...

splitting ... loop 600
Done
split is not yet balanced [60, 68, 55, 64, 71, 87, 60, 66].  splitting further ...

splitting ... l

In [15]:
print(f'train {train_props}')
print(f'test {test_props}')
print(f'dev {dev_props}')

train [58, 58, 62, 58, 54, 62, 59, 57]
test [20, 20, 21, 26, 29, 8, 23, 19]
dev [21, 20, 16, 15, 15, 29, 16, 23]


In [16]:
print(f" train balanced ? {_within_range(train_props)}")
print(f" test  balanced ? {_within_range(test_props)}")
print(f" dev   balanced ? {_within_range(dev_props)}")

 train balanced ? True
 test  balanced ? False
 dev   balanced ? False


In [17]:
 # train percentage 60%

for (k,v), (k2,v2) in zip(whole_dict.items(), train_dict.items()):
    if k2 == k:
        print(f" {k2 :{15}} | {v2 : {10}} | {k:{15}}| {v:{5}}  train percentage : {v2/v*100: .2f}")


 Analysis        |        654 | Analysis       |  1127  train percentage :  58.03
 DataCollection  |        114 | DataCollection |   196  train percentage :  58.16
 DataPreProcss   |        162 | DataPreProcss  |   261  train percentage :  62.07
 Modelling       |        118 | Modelling      |   203  train percentage :  58.13
 Programming     |         35 | Programming    |    64  train percentage :  54.69
 Simulation      |         15 | Simulation     |    24  train percentage :  62.50
 Stimulation     |         42 | Stimulation    |    71  train percentage :  59.15
 Visualization   |         51 | Visualization  |    89  train percentage :  57.30


# Pubmed_fulltext_bio

In [38]:
split_data(in_path_pubmed, out_path_pubmed)

Done


In [37]:

in_path_pubmed = '/home/beck/Desktop/SoMeNLP/data/Pubmed_fulltext_bio/'
out_path_pubmed = '/home/beck/Desktop/SoMeNLP/data/Pubmed_fulltext/'

split_data(in_path_pubmed, out_path_pubmed)

# pubmed bio   train , test , dev

pubmed_path_whole = '/home/beck/Desktop/SoMeNLP/data/Pubmed_fulltext_bio/'
pubmed_path_train = "/home/beck/Desktop/SoMeNLP/data/Pubmed_fulltext/Pubmed_fulltext_bio_train/" 
pubmed_path_test = "/home/beck/Desktop/SoMeNLP/data/Pubmed_fulltext/Pubmed_fulltext_bio_test/" 
pubmed_path_dev = "/home/beck/Desktop/SoMeNLP/data/Pubmed_fulltext/Pubmed_fulltext_bio_devel/"
    
    
#calculate the distribution of class_labels for pubmed
pubmed_whole_dict, pubmed_train_dict, pubmed_test_dict, pubmed_dev_dict = train_test_dev(pubmed_path_whole, 
                                                                                         pubmed_path_train, 
                                                                                         pubmed_path_test, 
                                                                                         pubmed_path_dev)


# calculate the proportion for each class label (8 - purposes)
pubmed_train_props, pubmed_dev_props, pubmed_test_props = __proportion_calculator(pubmed_whole_dict, 
                                                                                  pubmed_train_dict, 
                                                                                  pubmed_test_dict, 
                                                                                  pubmed_dev_dict)

pubmed_train_split_result_ = _within_range(pubmed_train_props)
pubmed_test_split_result_ = _within_range_pubmed(pubmed_test_props)
pubmed_dev_split_result_ = _within_range_pubmed(pubmed_dev_props)

bool_ = (pubmed_train_split_result_) and (pubmed_test_split_result_) and (pubmed_dev_split_result_)

i = 0

while not(pubmed_train_split_result_):
    
    print(f'splitting ... loop {i}')
    
    split_data(in_path_pubmed, out_path_pubmed)
    
    i += 1
    
    
    #calculate the new distribution 
    pubmed_whole_dict, pubmed_train_dict, pubmed_test_dict, pubmed_dev_dict = train_test_dev(pubmed_path_whole, 
                                                                                             pubmed_path_train, 
                                                                                             pubmed_path_test, 
                                                                                             pubmed_path_dev)
    
    
    # calculate the new proportion
    pubmed_train_props, pubmed_dev_props, pubmed_test_props = __proportion_calculator(pubmed_whole_dict, 
                                                                                      pubmed_train_dict, 
                                                                                      pubmed_test_dict, 
                                                                                      pubmed_dev_dict)
    #print('is the split fair ?')
    pubmed_split_result = _within_range(pubmed_train_props)
    
    
    
    if pubmed_split_result:
        
        print('pubmed split is balanced. stopped splitting')
        
    else:
        
        shutil.rmtree(pubmed_path_train, ignore_errors=False, onerror=None)
        shutil.rmtree(pubmed_path_test, ignore_errors=False, onerror=None)
        shutil.rmtree(pubmed_path_dev, ignore_errors=False, onerror=None)
        
        print(f'pubmed split is not yet balanced {pubmed_train_props}.  splitting further ...')
        print()
        
        
    
    #break

Done
splitting ... loop 0
Done
pubmed split is not yet balanced [73, 94, 62, 66, 93, 100, 96].  splitting further ...

splitting ... loop 1
Done
pubmed split is not yet balanced [44, 66, 32, 37, 93, 25, 46].  splitting further ...

splitting ... loop 2
Done
pubmed split is not yet balanced [59, 74, 66, 62, 93, 50, 50].  splitting further ...

splitting ... loop 3
Done
pubmed split is not yet balanced [65, 42, 68, 70, 93, 25, 60].  splitting further ...

splitting ... loop 4
Done
pubmed split is not yet balanced [55, 60, 47, 29, 93, 25, 64].  splitting further ...

splitting ... loop 5
Done
pubmed split is not yet balanced [52, 60, 62, 83, 87, 50, 50].  splitting further ...

splitting ... loop 6
Done
pubmed split is not yet balanced [58, 57, 63, 62, 100, 75, 82].  splitting further ...

splitting ... loop 7
Done
pubmed split is not yet balanced [44, 36, 41, 37, 87, 75, 46].  splitting further ...

splitting ... loop 8
Done
pubmed split is not yet balanced [64, 72, 81, 54, 100, 75, 75].

Done
pubmed split is not yet balanced [68, 40, 63, 91, 6, 75, 64].  splitting further ...

splitting ... loop 75
Done
pubmed split is not yet balanced [61, 56, 77, 37, 87, 50, 71].  splitting further ...

splitting ... loop 76
Done
pubmed split is not yet balanced [73, 62, 78, 79, 100, 75, 82].  splitting further ...

splitting ... loop 77
Done
pubmed split is not yet balanced [64, 76, 74, 66, 6, 100, 89].  splitting further ...

splitting ... loop 78
Done
pubmed split is not yet balanced [53, 74, 50, 58, 93, 75, 64].  splitting further ...

splitting ... loop 79
Done
pubmed split is not yet balanced [64, 57, 56, 50, 100, 75, 35].  splitting further ...

splitting ... loop 80
Done
pubmed split is not yet balanced [60, 44, 70, 29, 93, 50, 53].  splitting further ...

splitting ... loop 81
Done
pubmed split is not yet balanced [68, 72, 55, 62, 93, 25, 35].  splitting further ...

splitting ... loop 82
Done
pubmed split is not yet balanced [46, 34, 33, 41, 6, 25, 35].  splitting further .

Done
pubmed split is not yet balanced [40, 50, 33, 33, 93].  splitting further ...

splitting ... loop 147
Done
pubmed split is not yet balanced [58, 82, 60, 66, 87, 50, 71].  splitting further ...

splitting ... loop 148
Done
pubmed split is not yet balanced [66, 78, 63, 58].  splitting further ...

splitting ... loop 149
Done
pubmed split is not yet balanced [46, 52, 44, 50, 100, 75, 50].  splitting further ...

splitting ... loop 150
Done
pubmed split is not yet balanced [67, 76, 60, 87, 93, 25, 32].  splitting further ...

splitting ... loop 151
Done
pubmed split is not yet balanced [62, 57, 70, 45, 93, 50, 57].  splitting further ...

splitting ... loop 152
Done
pubmed split is not yet balanced [60, 60, 56, 70, 100, 75, 89].  splitting further ...

splitting ... loop 153
Done
pubmed split is not yet balanced [49, 54, 70, 54].  splitting further ...

splitting ... loop 154
Done
pubmed split is not yet balanced [56, 62, 54, 95, 100, 75, 50].  splitting further ...

splitting ... loo

Done
pubmed split is not yet balanced [67, 54, 67, 66, 93, 25, 32].  splitting further ...

splitting ... loop 221
Done
pubmed split is not yet balanced [52, 32, 59, 87, 6].  splitting further ...

splitting ... loop 222
Done
pubmed split is not yet balanced [49, 78, 59, 79].  splitting further ...

splitting ... loop 223
Done
pubmed split is not yet balanced [65, 66, 75, 75, 12, 75, 96].  splitting further ...

splitting ... loop 224
Done
pubmed split is not yet balanced [59, 62, 77, 54, 100, 75, 92].  splitting further ...

splitting ... loop 225
Done
pubmed split is not yet balanced [58, 42, 58, 91, 12, 75, 42].  splitting further ...

splitting ... loop 226
Done
pubmed split is not yet balanced [42, 46, 47, 29, 6, 75, 50].  splitting further ...

splitting ... loop 227
Done
pubmed split is not yet balanced [43, 54, 39, 33].  splitting further ...

splitting ... loop 228
Done
pubmed split is not yet balanced [48, 48, 55, 58, 93, 100, 57].  splitting further ...

splitting ... loop 2

Done
pubmed split is not yet balanced [63, 40, 64, 66, 93, 75, 67].  splitting further ...

splitting ... loop 294
Done
pubmed split is not yet balanced [61, 70, 43, 50, 6, 50, 60].  splitting further ...

splitting ... loop 295
Done
pubmed split is not yet balanced [72, 70, 72, 45, 100, 25, 64].  splitting further ...

splitting ... loop 296
Done
pubmed split is not yet balanced [65, 70, 45, 62, 12, 50, 71].  splitting further ...

splitting ... loop 297
Done
pubmed split is not yet balanced [61, 60, 72, 62, 12, 75, 60].  splitting further ...

splitting ... loop 298
Done
pubmed split is not yet balanced [42, 52, 32, 45, 6, 50, 53].  splitting further ...

splitting ... loop 299
Done
pubmed split is not yet balanced [56, 54, 75, 66, 12, 50, 50].  splitting further ...

splitting ... loop 300
Done
pubmed split is not yet balanced [53, 57, 51, 62, 12, 75, 71].  splitting further ...

splitting ... loop 301
Done
pubmed split is not yet balanced [46, 50, 36, 41, 12, 100, 82].  splitting f

...

Done
pubmed split is not yet balanced [58, 52, 54, 62, 12, 100, 71].  splitting further ...

splitting ... loop 14943
Done
pubmed split is not yet balanced [81, 62, 82, 91, 12, 50, 64].  splitting further ...

splitting ... loop 14944
Done
pubmed split is not yet balanced [63, 64, 60, 41, 12, 50, 60].  splitting further ...

splitting ... loop 14945
Done
pubmed split is not yet balanced [66, 42, 59, 75].  splitting further ...

splitting ... loop 14946
Done
pubmed split is not yet balanced [53, 66, 55, 45, 12, 75, 82].  splitting further ...

splitting ... loop 14947
Done
pubmed split is not yet balanced [59, 48, 68, 54, 93, 75, 71].  splitting further ...

splitting ... loop 14948
Done
pubmed split is not yet balanced [69, 57, 71, 62, 93, 100, 85].  splitting further ...

splitting ... loop 14949
Done
pubmed split is not yet balanced [69, 66, 78, 62, 93, 100, 82].  splitting further ...

splitting ... loop 14950
Done
pubmed split is not yet balanced [51, 44, 51, 70, 93, 50, 50].  spli

Done
pubmed split is not yet balanced [53, 64, 55, 33, 100, 50, 75].  splitting further ...

splitting ... loop 15015
Done
pubmed split is not yet balanced [61, 70, 56, 50, 6, 50, 35].  splitting further ...

splitting ... loop 15016
Done
pubmed split is not yet balanced [66, 74, 68, 45, 100, 50, 46].  splitting further ...

splitting ... loop 15017
Done
pubmed split is not yet balanced [63, 57, 66, 58, 6, 25, 28].  splitting further ...

splitting ... loop 15018
Done
pubmed split is not yet balanced [50, 46, 60, 54, 93, 25, 39].  splitting further ...

splitting ... loop 15019
Done
pubmed split is not yet balanced [65, 96, 87, 62, 93, 100, 92].  splitting further ...

splitting ... loop 15020
Done
pubmed split is not yet balanced [50, 64, 33, 41, 6].  splitting further ...

splitting ... loop 15021
Done
pubmed split is not yet balanced [55, 54, 63, 50, 93, 100, 60].  splitting further ...

splitting ... loop 15022
Done
pubmed split is not yet balanced [71, 50, 77, 75, 87, 50, 50].  sp

Done
pubmed split is not yet balanced [54, 78, 47, 58, 12, 75, 64].  splitting further ...

splitting ... loop 15086
Done
pubmed split is not yet balanced [74, 92, 82, 79, 93, 75, 75].  splitting further ...

splitting ... loop 15087
Done
pubmed split is not yet balanced [46, 48, 35, 50, 6, 25, 42].  splitting further ...

splitting ... loop 15088
Done
pubmed split is not yet balanced [55, 66, 71, 45, 100, 50, 46].  splitting further ...

splitting ... loop 15089
Done
pubmed split is not yet balanced [62, 50, 43, 54, 87, 25, 42].  splitting further ...

splitting ... loop 15090
Done
pubmed split is not yet balanced [55, 66, 55, 54, 6, 75, 89].  splitting further ...

splitting ... loop 15091
Done
pubmed split is not yet balanced [66, 52, 63, 75, 93, 75, 85].  splitting further ...

splitting ... loop 15092
Done
pubmed split is not yet balanced [54, 56, 43, 70, 12, 100, 85].  splitting further ...

splitting ... loop 15093
Done
pubmed split is not yet balanced [74, 66, 68, 75, 12, 100, 

Done
pubmed split is not yet balanced [70, 42, 54, 75, 93, 25, 28].  splitting further ...

splitting ... loop 15158
Done
pubmed split is not yet balanced [58, 62, 55, 87].  splitting further ...

splitting ... loop 15159
Done
pubmed split is not yet balanced [57, 34, 43, 33, 12, 25, 39].  splitting further ...

splitting ... loop 15160
Done
pubmed split is not yet balanced [59, 62, 75, 79, 12, 50, 71].  splitting further ...

splitting ... loop 15161
Done
pubmed split is not yet balanced [60, 57, 74, 37, 6, 50, 60].  splitting further ...

splitting ... loop 15162
Done
pubmed split is not yet balanced [54, 62, 52, 33, 100, 50, 64].  splitting further ...

splitting ... loop 15163
Done
pubmed split is not yet balanced [72, 76, 60, 58, 12, 75, 89].  splitting further ...

splitting ... loop 15164
Done
pubmed split is not yet balanced [71, 76, 72, 50, 6, 75, 67].  splitting further ...

splitting ... loop 15165
Done
pubmed split is not yet balanced [50, 42, 48, 25, 12, 50, 39].  splittin

KeyboardInterrupt: 

In [18]:
print(f'pubmed train {pubmed_train_props}')
print(f'pubmed test {pubmed_test_props}')
print(f'pubmed dev {pubmed_dev_props}')

pubmed train [60, 56, 58, 54]
pubmed test [19, 30, 18, 25, 6, 25, 21]
pubmed dev [19, 14, 22, 20, 93, 50, 17]


In [19]:
print(f" pubmed train balanced ? {_within_range(pubmed_train_props)}")
print(f" pubmed test  balanced ? {_within_range(pubmed_test_props)}")
print(f" pubmed dev   balanced ? {_within_range(pubmed_dev_props)}")

 pubmed train balanced ? True
 pubmed test  balanced ? False
 pubmed dev   balanced ? False


In [20]:
pubmed_whole_dict

{'Analysis': 309,
 'DataCollection': 50,
 'DataPreProcss': 74,
 'Modelling': 24,
 'Programming': 16,
 'Simulation': 4,
 'Visualization': 28}

In [21]:
pubmed_train_dict

{'Analysis': 188,
 'DataCollection': 28,
 'DataPreProcss': 43,
 'Modelling': 13,
 'Simulation': 1,
 'Visualization': 17}

In [22]:
pubmed_test_dict

{'Analysis': 60,
 'DataCollection': 15,
 'DataPreProcss': 14,
 'Modelling': 6,
 'Programming': 1,
 'Simulation': 1,
 'Visualization': 6}

In [23]:
pubmed_dev_dict

{'Analysis': 61,
 'DataCollection': 7,
 'DataPreProcss': 17,
 'Modelling': 5,
 'Programming': 15,
 'Simulation': 2,
 'Visualization': 5}

SyntaxError: invalid syntax (1556483325.py, line 1)